In [1]:
library("DBI")
library("dbplyr")
library("dplyr")

con <- DBI::dbConnect(
    RPostgres::Postgres(),
    host = "localhost",
    port = 5432,
    dbname = "knock100",
    user = "guest",
    password = "guest"
)

customer_tbl <- dplyr::tbl(con, "customer")
category_tbl <- dplyr::tbl(con, "category")
product_tbl <- dplyr::tbl(con, "product")
receipt_tbl <- dplyr::tbl(con, "receipt")
store_tbl <- dplyr::tbl(con, "store")
geo_tbl <- dplyr::tbl(con, 'geocode')
DBI::dbListTables(con)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:dbplyr’:

    ident, sql


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


[1] "customer" "category" "geocode"  "product"  "receipt"  "store"

### R-061:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(log_amount = log10(amount)) %>%
head

### R-062:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE )) %>%
mutate(log_amount = log(amount)) %>%
head

### R-063:
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [ ]:
product <- tbl(con, 'product')
product %>%
transmute(product_cd, unit_price, unit_cost, unit_profit = unit_price - unit_cost) %>%
head

### R-064:
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(avg_profit_rate = mean(unit_price - unit_cost, na.rm = TRUE)/mean(unit_price)) %>%
summarise(avg_profit_rate = min(avg_profit_rate))

### R-065:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = trunc(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-066:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = round(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-067:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = ceiling(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-068:
商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(price_taxin = trunc(1.1 * unit_price)) %>%
select(unit_price, price_taxin) %>%
head

### R-069:
レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [ ]:
receipt_tbl %>%
inner_join(product_tbl, by = 'product_cd') %>% 
group_by(customer_id) %>%
summarise(
	amount_canned = sum(if_else(category_major_cd == '07',amount, 0), na.rm = TRUE),
	amount_total = sum(amount)
) %>%
filter(amount_canned > 0) %>%
mutate(canned_ratio = as.numeric(amount_canned) / amount_total) %>%
head(10)

### R-070:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(elapsed_days = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% lubridate::days(1)) %>%
    head()

In [ ]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(interval = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd))) %>%
    head()

### R-071:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
# work in local
library(lubridate)

compare_date_local <- cul %>%
select(customer_id, application_date) %>%
inner_join(select(rel, customer_id, sales_ymd), by='customer_id') %>%
transmute(customer_id, sales_ymd = ymd(sales_ymd), application_date = ymd(application_date))

compare_date_local %>% 
mutate(elapsed_months = interval(application_date, sales_ymd) %/% months(1)) %>%
head

### R-072:
レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
customer_tbl %>%
inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by='customer_id') %>%
select(customer_id, sales_ymd, application_date) %>%
collect() %>%
mutate(elapsed_years = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% months(1)) %>%
head

In [ ]:
# work in local
compare_date_local %>%
mutate(elapsed_years = interval(application_date, sales_ymd) %/% years(1)) %>%
head

### R-073:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
# work in local
compare_date_local %>%
mutate(elapsed_epoch = (sales_ymd - application_date) %>% as.numeric(units = 'secs')) %>%
head

In [ ]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(elapsed_months = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% seconds(1)) %>%
    head()

### R-074:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
receipt_tbl %>%
select(sales_ymd) %>%
collect() %>%
mutate(
	elapsed_weekdays = lubridate::ymd(sales_ymd) %>% lubridate::wday(week_start = 1) -1,
	last_monday = lubridate::ymd(sales_ymd) - elapsed_weekdays
	) %>%
	head

In [ ]:
library(lubridate)
re %>%
data.frame %>%
mutate(
	sales_ymd_d = sales_ymd %>% ymd,
	elapsed_weekdays = sales_ymd_d %>% wday(week_start = 1)-1,
	last_monday = sales_ymd_d - elapsed_weekdays
	) %>%
	head

### R-075:
顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [ ]:
customer_tbl %>% slice_sample(n=customer_tbl %>% count() %>% collect() %>% as.numeric() %/% 100) %>% head

### R-076:
顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [ ]:
sampled_customer <- customer_tbl %>% 
	group_by(gender_cd) %>%
	collect() %>%
	dplyr::slice_sample(prop = 0.1)
sampled_customer %>% nrow() %>% as.numeric()

### R-077:
レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。


In [ ]:
receipt_tbl %>%
group_by(customer_id) %>%
summarise(log_amount = log(sum(amount, na.rm = TRUE))) %>%
mutate(mean = mean(log_amount, na.rm = TRUE), sd = sd(log_amount,na.rm = TRUE)) %>%
filter(!(log_amount <= mean+3*sd)) %>%
head

### R-078:
レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [ ]:
# assume you are on a local machine
library(stringr)

receipt_tbl %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
collect() %>%
filter(
	amount %>%
	{. > quantile(., probs = 0.25) + 1.5*IQR(.) | . < quantile(., probs = 0.25) - 1.5*IQR(.)}
) %>% head

In [ ]:
# assume you are on a local machine
library(stringr)

receipt_tbl %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
collect() %>%
filter(
	amount %>%
	{. > quantile(., probs = 0.25) + 1.5*IQR(.) | . < quantile(., probs = 0.25) - 1.5*IQR(.)}
) %>% head

In [ ]:
# assume you are on a local machine
rel <- DBI::dbReadTable(con, "receipt")
library(stringr)

rel %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(iqr = quantile(amount)[4] - quantile(amount)[2]) %>%
mutate(thre_lower = quantile(amount)[2] - 1.5*iqr, thre_upper = quantile(amount)[4] + 1.5*iqr) %>%
filter(amount < thre_lower | thre_upper < amount) %>%
head

### R-079:
商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [14]:
product_tbl %>% 
summarise(across(.cols = everything(), .fns = ~ if_else(is.na(.x), 1, 0) %>% sum(), .names = '{.col}_NA'))

# Source:   lazy query [?? x 6]
# Database: postgres [guest@localhost:5432/knock100]
  product_cd_NA category_major_… category_medium… category_small_… unit_price_NA
          <dbl>            <dbl>            <dbl>            <dbl>         <dbl>
1             0                0                0                0             7
# … with 1 more variable: unit_cost_NA <dbl>

### R-080:
商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [ ]:
# missing data
product_tbl %>% filter(if_any(everything(), ~ is.na(.))) %>% head

In [ ]:
# complete data
product_tbl %>% filter(if_all(-everything(), ~ !is.na(.))) %>% head

In [ ]:
# complete.cases does not works on tbl
product_tbl %>% collect() %>%
filter(complete.cases(.)) %>% nrow()